In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
# Step 1: Load and preprocess CIFAR data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [4]:
# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [5]:
# Step 2: Create data generators for training and testing
datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create a generator for training data
train_gen = datagen.flow(x_train, y_train, batch_size=32)

# Create a generator for validation/test data
val_gen = datagen.flow(x_test, y_test, batch_size=32)

In [6]:
# Step 3: Load VGG16 model without the top layer (for transfer learning)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the layers of VGG16 to avoid training them
base_model.trainable = False

# Step 4: Build the final model
model = models.Sequential()

# Add the base VGG16 model
model.add(base_model)

# Add custom layers on top of the VGG16 base model
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))  # Dropout for regularization
model.add(layers.Dense(10, activation='softmax'))  # 10 output classes for CIFAR-10


In [7]:
# Step 5: Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
# Step 6: Train the model using the generators
history = model.fit(train_gen, epochs=5, validation_data=val_gen)


Epoch 1/5


C:\Users\vinot\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 256s 163ms/step - accuracy: 0.0994 - loss: 2.3157 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 318s 203ms/step - accuracy: 0.1008 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 326s 209ms/step - accuracy: 0.1007 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 356s 228ms/step - accuracy: 0.1008 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 386s 247ms/step - accuracy: 0.0977 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026


In [10]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc * 1000:.2f}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 184s 587ms/step - accuracy: 0.0936 - loss: 2.3151
Test Accuracy: 90.50%
